# DCGAN
# Deep Convolutional Generative Adversarial Network

## Generative Adversarial Network

Las gan se basan en un punto de vista de teoría de juegos. Al contrario de una red neuronal convencional. En las gan se utilizan dos redes neuronales que reciben el nombre de **Generador** y **Discriminador**. Dichas redes están en constante competencia entre ellas.

#### [1] Generative Adversarial Networks
##### Ian J. Goodfellow, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville, Yoshua Bengio

We propose a new framework for estimating generative models via an adversarial process, in which we simultaneously train two models: a generative model G that captures the data distribution, and a discriminative model D that estimates the probability that a sample came from the training data rather than G. The training procedure for G is to maximize the probability of D making a mistake. This framework corresponds to a minimax two-player game. In the space of arbitrary functions G and D, a unique solution exists, with G recovering the training data distribution and D equal to 1/2 everywhere. In the case where G and D are defined by multilayer perceptrons, the entire system can be trained with backpropagation. There is no need for any Markov chains or unrolled approximate inference networks during either training or generation of samples. Experiments demonstrate the potential of the framework through qualitative and quantitative evaluation of the generated samples.

![GAN Diagram](img/1_XKanAdkjQbg1eDDMF2-4ow.png)

En esta distribución, la red generadora intenta generar imágenes que logren engañar al discriminador haciendole creer que las imágenes son originales. El discriminador por su parte trata de no ser engañado e intenta distinguir si las imágenes fueron originales o no.

El generador recibe como entrada un vector de numeros aleatorios (que llamaremos z) y lo transforma en datos que desea imitar.

El discriminador recibo como input tanto los datos reales (x) como los generados por el generador (G(z)) y computa la probabilidad de que esa entrada sea real.

### Objective Function

El discriminador trata de maximizar la función (gradient ascent) mientras que el discriminador trata de minimizarla (gradient descent)

![Objective Function](img/1_FbQLpEVQKsMSK-c7_5KcWw.png)

En la siguiente imágen, el primer término corresponde a que los datos reales tengan un alto valor mientras que motiva a los datos generados G(z) que sean ranqueados con una probabilidad baja.

In [ ]:
self.D, self.D_logits = self.discriminator(inputs, self.y, reuse=False)
self.D_, self.D_logits_ = self.discriminator(self.G, self.y, reuse=True)

# p * -tf.log(q) + (1 - p) * -tf.log(1 - q), q es el primer parámetro y p el segundo
self.d_loss_real = tf.reduce_mean(sigmoid_cross_entropy_with_logits(self.D_logits, tf.ones_like(self.D)))
self.d_loss_fake = tf.reduce_mean(sigmoid_cross_entropy_with_logits(self.D_logits_, tf.zeros_like(self.D_)))

self.d_loss = self.d_loss_real + self.d_loss_fake

Sin embargo, al aplicar esta ecuación, el generador no funciona tan bien. Esto sucede porque cuando una imagen es generada es probable que lo clasifique como falso. El gradiende tiende a ser bastante plano y dificulta que el modelo aprenda correctamente. Por dicho motivo se cambia la función del generador por la siguiente:

![Generator Objective Function](img/1_ZHKnky7Pzi5OvPlUIZhDjg.png)

Es decir, que en lugar de minimizar la probabilidad de que el discriminador tenga razón, maximiza el *likelyhood* de que el discriminador se equivoque.

In [ ]:
self.g_loss = tf.reduce_mean(sigmoid_cross_entropy_with_logits(self.D_logits_, tf.ones_like(self.D_)))

### Generador

![Generator](img/gernerator.png)

La entrada del generador es una entrada aleatoria denominada *latent sample*. El generador toma esa entrada y la convierte en la imagen generada.

Resulta evidente que sin entrenamiento, la salida de la red será ruido sin significado.

### Discriminador 

El discriminador recibe una imágen y dice si la misma fue real (1) o no (0).

![Generator](img/disc.png)

### Entrenamiento
    1. Poner el discriminador en modo de entrenamiento
    2. Entrenar el discriminador tanto con imágenes generadas como reales
    3. Poner el discriminador en modo de no entrenamiento
    4. Entrenar el generador como parte de la GAN
    5. Repetir paso 1

#### A tener en cuenta

Si el discriminador entrena mucho más rápido que el generador, el generador nunca logra engañar al discriminador. Lo mismo aplica para el otro caso en donde el discriminador termina no pudiendo clasificar apropiadamente. Se debe tener cuidado para lograr que ambos logren entrenarse a un ritmo similar.

## Convolutional Neural Networks

CNNs son especialmente útiles para clasificación y reconocimiento de imágenes.

CNN poseen a grandes rasgos dos componentes principales:
    1. Las capas ocultas (feature extraction)
    2. Clasificación
    
![Generator](img/1_NQQiyYqJJj4PSYAeWvxutg.png)

### Feature Extraction

En este componente se realizan operaciones de **convolucion** y **pooling** en las cuales los patrones son detectados.

Si se buscara reconocer una zebra por ejemplo, esta etapa reconocería las rayas, dos oídos y cuatro patas.

#### Convolución

En la convolución se dice que se convoluciona la imagen de entrada con un **kernel** o **filtro** para generar un **feature map**. Para realizar la convolución se mueve el filtro sobre la imagen de entrada multiplicando y sumando el resultado en el *feature map*. 

En la siguiente imágen peude observarse claramente cómo se realiza dicha operación.
![conv](img/1_VVvdh-BUKFh2pwDD0kPeRA@2x.gif)

En la práctica se realizan numerosas convoluciones sobre la entrada usando diferentes filtros. Esto genera numerosos *feature maps* los cuales se los junta para obtener la salida final de la capa de convolución.

#### Función de activación

Como en cualquier otra red neuronal, se usa una **función de activación** para que la salida sea no lineal. Por ejemplo la función ReLU (Rectified Linear Units - https://github.com/Kulbear/deep-learning-nano-foundation/wiki/ReLU-and-Softmax-Activation-Functions)

$$ f(x) = max(x, 0) $$

#### Stride

Stride se le llama al *paso* (cantidad de pixels) que el filtro debe moverse a cada iteración. Usualmente es 1. Aumentando dicho número puede reducirse el overlap.

![stride](img/0_iqNdZWyNeCr5tCkc_.gif)

#### Padding

El tamaño del *feature map* es SIEMPRE menor que el input. Es por eso que se debe usar **padding**.

Una capa de pixels nulos (valor cero) se agrega al input, rodeando al mismo de ceros y aumentando de esa forma su tamaño. De esta forma se logra que no se reduzca el *feature map*. El ejemplo de stride superior incluye un padding representado por los cuadrados de linea punteada.

El padding además, mejora la performance y se asegura que el tamaño del kernel y del stride sean coherentes con la entrada.

#### Pooling

Luego de una capa de convolución, es común agregar una capa de **pooling**. Su función es reducir continuamente las dimensiones reduciendo la complejidad de la red.

Lo mismo decrementa el tiempo de entrenamiento y reduce el overfitting.

##### Max Pooling

El modo más común de pooling se llama **max pooling** el cual toma el máximo valor de cada ventana. En la siguiente figura se muestra un ejemplo de max pooling:

![stride](img/1_vbfPq-HvBCkAcZhiSTZybg.png)

#### Resumen

Al usar una CNN hay 4 hiperparámetros importantes entre los cuales decidir:

1. Kernel size
2. Filter count (cuantos filtros usar)
3. Stride
4. Padding

Visualización de una capa convolucional:

![stride](img/1__34EtrgYk6cQxlJ2br51HQ.gif)

### Classification

Ocurre luego de las capas de convolución y pooling.

Clasifica como una red convencional sobre los patrones obtenidos.

La parte de clasificación simplemente consiste en una red fully connected convirtiendo la matriz 3D (o 2D si es grayscale) en un vector 1D.

La red se entrena igual que cualquier otra red, usando backpropagation / gradient descent.

## DCGAN

![Generator](img/1_39Nnni_nhPDaLu9AnTLoWw.png)
https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39

#### [2] Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks
##### Alec Radford, Luke Metz, Soumith Chintala

In recent years, supervised learning with **convolutional networks (CNNs)** has seen huge adoption in computer vision applications. Comparatively, unsupervised learning with CNNs has received less attention. In this work we hope to help bridge the **gap between the success of CNNs for supervised learning and unsupervised learning.** We introduce a class of CNNs called deep convolutional generative adversarial networks (DCGANs), that have certain architectural constraints, and demonstrate that they are a strong candidate for unsupervised learning. Training on various image datasets, we show convincing evidence that our deep convolutional adversarial pair learns a hierarchy of representations from object parts to scenes in both the generator and discriminator. Additionally, we use the learned features for novel tasks - demonstrating their applicability as general image representations.

## Código

La mayor parte del código se basó en https://github.com/carpedm20/DCGAN-tensorflow

### main.py

Se importan los parámetros mediante la linea de comando gracias al módulo *flags* de TensorFlow. 

De esta forma se permite que un usuario utilice el programa de un modo sencillo sin necesidad de abrir ni modificar el código.

**Nota:** Hay un bug en el código, es necesario usar la opción crop obligatoriamente.

In [6]:
import os
import numpy as np
import tensorflow as tf

flags = tf.app.flags
flags.DEFINE_integer("epoch", 25, "Epoch to train [25]")
flags.DEFINE_float("learning_rate", 0.0002, "Learning rate of for adam [0.0002]")
flags.DEFINE_float("beta1", 0.5, "Momentum term of adam [0.5]")
flags.DEFINE_float("train_size", np.inf, "The size of train images [np.inf]")
flags.DEFINE_integer("batch_size", 64, "The size of batch images [64]")
flags.DEFINE_integer("input_height", 108, "The size of image to use (will be center cropped). [108]")
flags.DEFINE_integer("input_width", None,
                     "The size of image to use (will be center cropped). If None, same value as input_height [None]")
flags.DEFINE_integer("output_height", 64, "The size of the output images to produce [64]")
flags.DEFINE_integer("output_width", None,
                     "The size of the output images to produce. If None, same value as output_height [None]")
flags.DEFINE_string("dataset", "anime-faces", "The name of dataset [celebA, mnist, lsun]")
flags.DEFINE_string("input_fname_pattern", "*.jpg", "Glob pattern of filename of input images [*]")
flags.DEFINE_string("checkpoint_dir", "checkpoint", "Directory name to save the checkpoints [checkpoint]")
flags.DEFINE_string("data_dir", "../data", "Root directory of dataset [data]")
flags.DEFINE_string("sample_dir", "samples", "Directory name to save the image samples [samples]")
flags.DEFINE_boolean("train", False, "True for training, False for testing [False]")
flags.DEFINE_boolean("crop", True, "True for training, False for testing [False]")
flags.DEFINE_boolean("visualize", False, "True for visualizing, False for nothing [False]")
flags.DEFINE_integer("generate_test_images", 100, "Number of images to generate during test. [100]")
FLAGS = flags.FLAGS

ModuleNotFoundError: No module named 'model'

In [ ]:
def main(_):
    pp.pprint(flags.FLAGS.__flags)                  # Muestro los parámetros

    if FLAGS.input_width is None:
        FLAGS.input_width = FLAGS.input_height      # Si no se especifica se asume imagen cuadrada
    if FLAGS.output_width is None:
        FLAGS.output_width = FLAGS.output_height

    if not os.path.exists(FLAGS.checkpoint_dir):    # Crea las carpetas si no existen
        os.makedirs(FLAGS.checkpoint_dir)
    if not os.path.exists(FLAGS.sample_dir):
        os.makedirs(FLAGS.sample_dir)

    run_config = tf.ConfigProto()
    run_config.gpu_options.allow_growth = True

    with tf.Session(config=run_config) as sess:
        dcgan = DCGAN(
            sess,
            input_width=FLAGS.input_width,
            input_height=FLAGS.input_height,
            output_width=FLAGS.output_width,
            output_height=FLAGS.output_height,
            batch_size=FLAGS.batch_size,
            sample_num=FLAGS.batch_size,
            z_dim=FLAGS.generate_test_images,
            dataset_name=FLAGS.dataset,
            input_fname_pattern=FLAGS.input_fname_pattern,
            crop=FLAGS.crop,
            checkpoint_dir=FLAGS.checkpoint_dir,
            sample_dir=FLAGS.sample_dir,
            data_dir=FLAGS.data_dir
        )
        show_all_variables()

        if FLAGS.train:
            dcgan.train(FLAGS)
        else:
            if not dcgan.load(FLAGS.checkpoint_dir)[0]:
                raise Exception("[!] Train a model first, then run test mode")

        # to_json("./web/js/layers.js", [dcgan.h0_w, dcgan.h0_b, dcgan.g_bn0],
        #                 [dcgan.h1_w, dcgan.h1_b, dcgan.g_bn1],
        #                 [dcgan.h2_w, dcgan.h2_b, dcgan.g_bn2],
        #                 [dcgan.h3_w, dcgan.h3_b, dcgan.g_bn3],
        #                 [dcgan.h4_w, dcgan.h4_b, None])

        # Below is codes for visualization
        OPTION = 1
        visualize(sess, dcgan, FLAGS, OPTION)

### model.py

Métodos principales:

#### 1. Constructores/Inicializadores

*Init:*

    1.  Guarda las variables dentro del objeto.
    2.  Batch Normalization.
    3.  Se fija si es grayscale o a color (para poner la cantidad de canales).
    4.  Verifica que el batch_size sea menor a la longitud total de los datos.
    5.  Llama a build model.

*build_model:*

    1.  Setea las dimensiones según la entrada *crop*.
        a.  Si crop entonces usa la salida como tamaño de entrada y corta las imágenes desde el centro.
    2.  Llama a los consctructores del generador y el discriminador.
    3.  Llama a los consctuctores del sampler.
    4.  Crea las funciones de costo para ambas redes.
    5.  Utiliza trainable_variables y lo divide en generador y discriminador para tener la lista de las variables a entrenar.
    6.  Inicializa un objeto de clase *Saver* para ir guardando los checkpoints.

*discriminator:*
    
    -  Crea la esctructura de la red del discriminador

*generator:*
    
    -  De forma análoga a discriminador, crea la estructura de la red
    -  Utiliza deconvolución en lugar de convolución

*sampler:*

#### 2. Entrenamiento

*train:*

    -  Adam Optimizer (parámetros beta para el decay del momentum)
    1.  Genera el ruido de entrada
    2.  Toma la cantidad de imágenes correspondiente al batch size
    3.  Intenta abrir un checkpoint
    4.  Comienza las epochs
        4.1.  Para cada epoch divide en test set según el batch size
        4.2.  Entrena D
        4.3.  Entrena G
        4.4.  Entrena G again (para que D_loss no llegue a cero)
    5.  Si hay que hacer un sample hace sample
    6.  Si checkpoint entonces guarda el checkpoint
        
    

#### 3. Checkpoint

*model_dir:*

    -  Obtiene el nombre de un directorio a utilizar para guardar el modelo según:
        1.  Nombre del dataset
        2.  batch_size
        3.  dimensiones de las imágenes de salida
        
*save:*

    -  Guarda en el directorio dado por checkpoint y "model_dir" la sesión (sess)

*load:*

    -  Abre la sesión guardada con "save"

## Resultados


## Posibles mejoras

### How to Train a GAN? Tips and tricks to make GANs work
https://github.com/soumith/ganhacks

### Dataset

https://danbooru.donmai.us/

#### Dataset no uniforme

    - Blanco y negro
    - Errores de recorte
    - Perfiles
    
[3] Dice: " [...] their (Danbooru) datasets suffer from high inter-image variance and noise. Due to the fact that image boards allow uploading of images highly different in style, domain, and quality [...]"

No solo utilizan una base de datos más homogenea sino que luego revisan todos los datos y eliminan un 4% de los falsos negativos (lo cual no se hizo en este trabajo).

<tr>
    <td> <img src="img/dataset/danbooru_901019_ed9e65500490e35ea9d892eb6a998ffb.png" alt="Drawing" /> </td>
    <td> <img src="img/dataset/danbooru_853258_6a2fae7e35d57e53db563434a1c550d1.png" alt="Drawing" /> </td>
    <td> <img src="img/dataset/danbooru_843704_8c4e83901fa17a685541fd67e218f2b0.png" alt="Drawing" /> </td>
    
    <td> <img src="img/dataset/danbooru_836273_d6147d08cf5521d625e729d93715b0fe.png" alt="Drawing" /> </td>
    <td> <img src="img/dataset/danbooru_486418_f991370eb8779ed66e1aa8d5f0996395.jpg" alt="Drawing" /> </td>
    <td> <img src="img/dataset/danbooru_454550_a81283d43c096498fdda8b332831a1b9.jpg" alt="Drawing" /> </td>
    
    <td> <img src="img/dataset/danbooru_444173_0066c72cc0066531084a288e1a4e45a7.jpg" alt="Drawing" /> </td>
    <td> <img src="img/dataset/danbooru_440907_0b1a026392412dcfddab3871cb412bff.jpg" alt="Drawing" /> </td>
    <td> <img src="img/dataset/danbooru_564654_5133896875a2d5952d035c573af0afb3.png" alt="Drawing" /> </td>
    </tr>

## Bibliografía:

#### [3] Towards the Automatic Anime Characters Creation with Generative Adversarial Networks

Automatic generation of facial images has been well studied after the Generative
Adversarial Network(GAN) came out. There exists some attempts applying the
GAN model to the problem of generating facial images of anime characters, but
none of the existing work gives a promising result. In this work, we explore the
training of GAN models specialized on an anime facial image dataset. We address
the issue from both the data and the model aspect, by collecting a more clean,
well-suited dataset and leverage proper, empirical application of DRAGAN. With
quantitative analysis and case studies we demonstrate that our efforts lead to a
stable and high-quality model. Moreover, to assist people with anime character
design, we build a website1 with our pre-trained model available online, which
makes the model easily accessible to general public.